# 02 - Gentle Roaster 

This notebook implements our most constructive CV critique model.

## Characteristics
- Generally friendly
- Focuses on  improvements & highlights strengths before weaknesses
- Uses lower temperature 0.3-0.5

---

In [13]:
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
import sys
sys.path.append('..')
import google.generativeai as genai

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

## Setup

- Loading API from config.py (only saved locally)

In [14]:
# Load API key from config.py
import sys
sys.path.append('..')
from config import GEMINI_API_KEY
genai.configure(api_key=GEMINI_API_KEY)
print("API key loaded from config.py")

API key loaded from config.py


## Load Data and Helper Functions

In [15]:
# Load dataset
df = pd.read_csv('../data/resume_data.csv')

# Load test CV indices
with open('../data/test_cv_indices.json', 'r') as f:
    test_data = json.load(f)
    test_cv_indices = test_data['indices']

print(f"Loaded {len(df)} resumes")
print(f"Test CVs: {test_cv_indices}")

Loaded 9544 resumes
Test CVs: [0, 1]


In [16]:
# CV formatting function (from EDA notebook)
def format_cv_for_llm(resume_row):
    """
    Format a resume row into a readable text for LLM processing.
    """
    cv_text = []
    
    if pd.notna(resume_row.get('career_objective')):
        cv_text.append(f"CAREER OBJECTIVE:\n{resume_row['career_objective']}")
    
    if pd.notna(resume_row.get('skills')):
        cv_text.append(f"\nSKILLS:\n{resume_row['skills']}")
    
    education_parts = []
    if pd.notna(resume_row.get('educational_institution_name')):
        education_parts.append(f"Institution: {resume_row['educational_institution_name']}")
    if pd.notna(resume_row.get('degree_names')):
        education_parts.append(f"Degree: {resume_row['degree_names']}")
    if pd.notna(resume_row.get('major_field_of_studies')):
        education_parts.append(f"Major: {resume_row['major_field_of_studies']}")
    if pd.notna(resume_row.get('passing_years')):
        education_parts.append(f"Year: {resume_row['passing_years']}")
    
    if education_parts:
        cv_text.append(f"\nEDUCATION:\n" + "\n".join(education_parts))
    
    work_parts = []
    if pd.notna(resume_row.get('professional_company_names')):
        work_parts.append(f"Company: {resume_row['professional_company_names']}")
    if pd.notna(resume_row.get('positions')):
        work_parts.append(f"Position: {resume_row['positions']}")
    if pd.notna(resume_row.get('start_dates')):
        work_parts.append(f"Period: {resume_row['start_dates']}")
        if pd.notna(resume_row.get('end_dates')):
            work_parts.append(f" to {resume_row['end_dates']}")
    if pd.notna(resume_row.get('responsibilities')):
        work_parts.append(f"Responsibilities:\n{resume_row['responsibilities']}")
    
    if work_parts:
        cv_text.append(f"\nWORK EXPERIENCE:\n" + "\n".join(work_parts))
    
    if pd.notna(resume_row.get('languages')):
        cv_text.append(f"\nLANGUAGES:\n{resume_row['languages']}")
    
    if pd.notna(resume_row.get('certification_skills')):
        cv_text.append(f"\nCERTIFICATIONS:\n{resume_row['certification_skills']}")
    
    return "\n".join(cv_text)

## Gentle Roaster Prompt Design

In [17]:
GENTLE_SYSTEM_PROMPT = """You are a kind and supportive career advisor who helps people improve their resumes.

Your approach:
1. Start with genuine positive observations about their CV
2. Provide constructive, specific feedback
3. Suggest concrete improvements
4. End with encouragement

Keep your feedback:
- Professional and friendly
- Specific and actionable
- Balanced (acknowledge strengths)
- Encouraging and motivating

Structure your response:
 STRENGTHS: What's working well
 AREAS FOR IMPROVEMENT: Specific suggestions
 ACTION ITEMS: Clear next steps
 FINAL THOUGHTS: Encouraging message
"""

def create_gentle_prompt(cv_text):
    """Create a gentle roasting prompt."""
    return f"""Please review this CV and provide constructive, encouraging feedback:

{cv_text}

Provide your feedback following the structure in the system prompt."""

## Temperature Tuning Experiments

We test different temperature settings to find the optimal setting for for "gentle" feedback.

In [18]:
def roast_cv(cv_text, temperature=0.4, model_name="gemini-2.0-flash"):
    """
    Generate CV critique using Gemini.
    
    Args:
        cv_text: Formatted CV text
        temperature: Controls randomness (0.0-1.0)
        model_name: Gemini model to use
    
    Returns:
        str: Generated critique
    """
    model = genai.GenerativeModel(
        model_name=model_name,
        generation_config=genai.GenerationConfig(
            temperature=temperature,
            top_p=0.95,
            top_k=40,
            max_output_tokens=1024,
        )
    )
    
    # Combine system prompt and user prompt
    full_prompt = f"{GENTLE_SYSTEM_PROMPT}\n\n{create_gentle_prompt(cv_text)}"
    
    response = model.generate_content(full_prompt)
    return response.text

# Test with first CV
test_cv = format_cv_for_llm(df.iloc[test_cv_indices[0]])
print("Test CV:")
print("="*80)
print(test_cv[:500] + "...")
print("="*80)

Test CV:
CAREER OBJECTIVE:
Big data analytics working and database warehouse manager with robust experience in handling all kinds of data. I have also used multiple cloud infrastructure services and am well acquainted with them. Currently in search of role that offers more of development.

SKILLS:
['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapreduce', 'Spark', 'Java', 'Machine Learning', 'Cloud', 'Hdfs', 'YARN', 'Core Java', 'Data Science', 'C++', 'Data Structures', 'DBMS', 'RDBMS', 'Informatica', 'Talend...


### Experiment 1: Low Temperature (0.3)
More focused and consistent

In [19]:
print(" Temperature: 0.3 (Most Conservative)")
print("="*80)
result_temp_03 = roast_cv(test_cv, temperature=0.3)
print(result_temp_03)
print("\n" + "="*80)

 Temperature: 0.3 (Most Conservative)
Okay, I'd be happy to review your CV and provide some constructive feedback! Here's my assessment:

**STRENGTHS:**

*   **Comprehensive Skills Section:** You've listed a good range of technical skills relevant to big data and cloud technologies. This gives a clear overview of your expertise.
*   **Relevant Work Experience:** Your experience as a Big Data Analyst at Coca-Cola is directly relevant to your career objective.
*   **Education:** Including your B.Tech degree provides a solid foundation for your technical skills.

**AREAS FOR IMPROVEMENT:**

*   **Career Objective:** The current objective is a bit generic. It could be more impactful by specifying the type of development role you're seeking and highlighting what makes you a strong candidate.
*   **Skills Section:** While comprehensive, it lacks context. Consider quantifying your proficiency level in each skill (e.g., "Proficient in Python for data analysis and scripting").
*   **Work Experi

### Experiment 2: Medium-Low Temperature (0.5)
More creative

In [20]:
print(" Temperature: 0.5 (Balanced)")
print("="*80)
result_temp_05 = roast_cv(test_cv, temperature=0.5)
print(result_temp_05)
print("\n" + "="*80)

 Temperature: 0.5 (Balanced)
Okay, I'd be happy to review your CV and provide some constructive feedback! Here's my assessment:

**STRENGTHS:**

*   You have a good range of technical skills listed, covering Big Data technologies, programming languages, and cloud platforms. This demonstrates a broad understanding of the field.
*   Listing your education with the institution, degree, major, and year provides a clear picture of your academic background.
*   The inclusion of your work experience at Coca-Cola is valuable, especially since it's relevant to the "Big Data Analyst" role.
*   Listing responsibilities gives potential employers an idea of what you did in your role.

**AREAS FOR IMPROVEMENT:**

*   **Career Objective:** The current objective is a bit generic. It could be more impactful by being more specific about the type of development role you're seeking and highlighting what you bring to the table.
*   **Skills:** While the list is comprehensive, it lacks context. Consider add

### Experiment 3: Medium Temperature (0.7)
Very creative

In [21]:
print(" Temperature: 0.7 (More Creative)")
print("="*80)
result_temp_07 = roast_cv(test_cv, temperature=0.7)
print(result_temp_07)
print("\n" + "="*80)

 Temperature: 0.7 (More Creative)
Okay, I'd be happy to review your CV and offer some constructive feedback. Here's my take:

**STRENGTHS:**

*   You've clearly identified a target role in your career objective.
*   You have a good range of technical skills listed, showcasing your expertise in Big Data, Cloud technologies, and programming languages.
*   You've included relevant keywords (like Hadoop, Spark, Azure) which can help your CV get noticed by Applicant Tracking Systems (ATS).
*   Listing your work experience, even with brief descriptions, provides a foundation for demonstrating your capabilities.

**AREAS FOR IMPROVEMENT:**

*   **Career Objective:** The career objective is a bit generic. It could be more impactful by specifying the type of development role you're seeking and hinting at what you can bring to a potential employer.
*   **Skills:** While the list is comprehensive, it lacks context. Consider quantifying your proficiency level in each skill (e.g., "Proficient in Py

## Select Optimal Temperature

Based on reading through the experiments above, we manually select the temperature that provides the best balance of:
- Professional tone, Specific feedback, Encouraging message and Consistency

**Recommended: 0.4 for gentle roasting**

In [22]:
# Set optimal temperature
OPTIMAL_TEMPERATURE = 0.4

print(f" Selected optimal temperature: {OPTIMAL_TEMPERATURE}")

 Selected optimal temperature: 0.4


## Summary

This notebook demonstrated:
1.  Gentle CV critique prompt design
2.  Temperature tuning experiments & selection 
3.  Generation of critiques for test CVs
4.  Saving results for comparison

---

## Next: 03_medium_roaster.ipynb
More direct and honest feedback with balanced criticism.